**Entraînement du Word2Vec avec Google Colaboratory**
Avant tout, assurez vous d'avoir plusieurs choses :

*   Google Colaboratory doit être synchonisé avec Google Drive

In [0]:
# Télécharge data_lelu_cleaned.txt
%%bash
if [ ! -f "/content/drive/My Drive/data_lelu_cleaned.txt" ]; then
  # Installation de LRZIP
  apt-get install lrzip
  # Téléchargement de l'archive
  wget https://github.com/LaraProject/notebooks/raw/master/data_lelu_cleaned.txt.lrz
  # Décompression
  lrunzip data_lelu_cleaned.txt.lrz
fi

In [0]:
# Assemblage des deux datasets
%%bash
if [ -f "/content/drive/My Drive/data_lelu_cleaned.txt" ]; then
  cat "/content/drive/My Drive/data_lelu_cleaned.txt" "/content/drive/My Drive/data_facebook_cleaned.txt" > data.txt
fi
if [ -f "/content/data_lelu_cleaned.txt" ]; then
  cat "/content/data_lelu_cleaned.txt" "/content/drive/My Drive/data_facebook_cleaned.txt" > data.txt
fi

In [0]:
from gensim.models import FastText

class MyIter(object):
  def __iter__(self):
    with open("data.txt", 'r') as f:
      count = 0
      for line in f.readlines() + ["<unk> "*100 + "\n"]:
        if (count % 2) == 0:
          sentence = line[11:-1].split()
        else:
          sentence = ["<start>"] + line[9:-1].split() + ["<end>"]
        yield sentence
        count += 1

In [0]:
# Construction du modèle
model = FastText(size=100, window=5, min_count=20, workers=2)
model.build_vocab(sentences=MyIter())
total_examples = model.corpus_count

In [0]:
# Entraînement
model.train(sentences=MyIter(), total_examples=total_examples, epochs=5)

In [0]:
# Sauvegarde
model.wv.save_word2vec_format("/content/drive/My Drive/word2vec_vectors.txt")
model.save("/content/drive/My Drive/word2vec_model.bin")

**Conservez précieusement les nouveaux fichiers apparus sur votre Drive**